# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet('goodsread/items.parquet')
events = pd.read_parquet('goodsread/events.parquet')

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
events.iloc[0].to_dict()

{'user_id': 1229132,
 'book_id': 22034,
 'started_at': datetime.date(2015, 7, 12),
 'read_at': datetime.date(2015, 7, 17),
 'is_read': True,
 'rating': 5,
 'is_reviewed': False,
 'started_at_month': datetime.date(2015, 7, 1)}

Разбиение выборки (задание 2 из 6)

In [5]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime('2017-08-01').date()

train_test_global_time_split_idx = (
    events["started_at"] < train_test_global_time_split_date
)

events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train['user_id'].drop_duplicates()
users_test = events_test['user_id'].drop_duplicates()

# количество пользователей, которые есть и в train, и в test
common_users = set(users_train) & set(users_test)

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


Идентифицируем холодных пользователей (задание 3 из 6)

In [6]:
cold_users = set(users_test) - common_users

print(len(cold_users))

2365


Топ-100 наиболее популярных книг с 2015 года и со средней оценкой не меньше 4 (задание 4 из 6)

In [7]:
top_pop_start_date = pd.to_datetime('2015-01-01').date()

item_popularity = (
    events_train
    .query('started_at >= @top_pop_start_date')
    .groupby(['book_id'])
    .agg(users=('user_id', 'nunique'), avg_rating=('rating', 'mean'))
    .reset_index()
)
item_popularity['popularity_weighted'] = (
    item_popularity['users'] * item_popularity['avg_rating']
)

In [20]:
# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(
    'popularity_weighted',
    ascending=False
)
# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = (
    item_popularity
    .query('avg_rating >= 4')
    #.sort_values('popularity_weighted', ascending=False)
    .head(100)
)

top_k_pop_items.head(3)

,book_id,users,avg_rating,popularity_weighted
32387,18007564,20207,4.321275,87320.0
32623,18143977,19462,4.290669,83505.0
30695,16096824,16770,4.301014,72128.0


Задание 5 из 6

In [21]:
# Для событий холодных пользователей добавляем данные из top100 (если есть)
cold_users_events_with_recs = (
    events_test[events_test['user_id'].isin(cold_users)]
    .merge(top_k_pop_items, on='book_id', how="left")
)

cold_users_events_with_recs.head(3)

,user_id,book_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,users,avg_rating,popularity_weighted
0,1361610,6900,2017-10-09,2017-10-13,True,4,False,2017-10-01,NaN,NaN,NaN
1,1361610,12555,2017-09-21,2017-10-11,True,3,False,2017-09-01,NaN,NaN,NaN
2,1361610,25899336,2017-09-12,2017-09-17,True,4,True,2017-09-01,4798.0,4.427261,21242.0


In [22]:
# Индексы событий НЕ из top100
cold_user_items_no_avg_rating_idx = (
    cold_users_events_with_recs['avg_rating'].isnull()
)
# События холодных пользователей с книгами из top100
cold_user_recs = (
    cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx]
    [['user_id', 'book_id', 'rating', 'avg_rating']]
)
cold_user_recs.head(3)

,user_id,book_id,rating,avg_rating
2,1361610,25899336,4,4.427261
5,1338996,16096824,5,4.301014
8,1338996,18692431,5,4.071454


In [23]:
len(cold_user_recs)

1912

In [24]:
len(cold_users_events_with_recs)

9672

In [25]:
cold_user_items_no_avg_rating_idx.sum()

7760

In [26]:
1- len(cold_user_recs) / len(cold_users_events_with_recs)

0.8023159636062862

Задание 6 из 6

In [27]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(
    cold_user_recs['rating'], 
    cold_user_recs['avg_rating'],
    squared=False
)
mae = mean_absolute_error(
    cold_user_recs['rating'],
    cold_user_recs['avg_rating']
)
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


Доп. код

In [28]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

# === Базовые подходы: коллаборативная фильтрация

# === Базовые подходы: контентные рекомендации

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков